## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-24-06-04-00 +0000,wsj,China Hits Japanese Firms With Export Bans,https://www.wsj.com/economy/global/china-slaps...
1,2026-02-24-06-02-26 +0000,bbc,Bowen: Why Ukraine remains defiant and does no...,https://www.bbc.com/news/articles/cd9g2ggq4epo...
2,2026-02-24-06-02-13 +0000,bbc,Russia soldiers tell BBC they saw fellow troop...,https://www.bbc.com/news/articles/cz7gw3l395ro...
3,2026-02-24-06-01-06 +0000,bbc,Ex-Philippine president Duterte on trial for c...,https://www.bbc.com/news/articles/c363xd6p1gxo...
4,2026-02-24-05-59-14 +0000,bbc,Landmark royal commission into antisemitism pr...,https://www.bbc.com/news/articles/c743dzvp73xo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2493/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
59,trump,50
87,mexico,25
90,cartel,24
85,violence,16
301,tariff,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
127,2026-02-23-22-42-00 +0000,wsj,The Trump administration is considering new na...,https://www.wsj.com/politics/policy/trump-cons...,130
304,2026-02-23-14-17-01 +0000,nypost,Expert’s chilling warning after killing of car...,https://nypost.com/2026/02/23/world-news/exper...,127
55,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...,119
41,2026-02-24-03-00-00 +0000,wsj,Beijing Sees Opening in U.S. Trade War After C...,https://www.wsj.com/world/china/beijing-sees-o...,115
33,2026-02-24-03-16-21 +0000,nyt,Drug Lord ‘El Mencho’ Was Killed in Mexico. Wh...,https://www.nytimes.com/2026/02/23/world/ameri...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
127,130,2026-02-23-22-42-00 +0000,wsj,The Trump administration is considering new na...,https://www.wsj.com/politics/policy/trump-cons...
304,127,2026-02-23-14-17-01 +0000,nypost,Expert’s chilling warning after killing of car...,https://nypost.com/2026/02/23/world-news/exper...
263,85,2026-02-23-17-28-59 +0000,nypost,Ex-British Ambassador to US Peter Mandelson ar...,https://nypost.com/2026/02/23/world-news/ex-br...
120,58,2026-02-23-22-57-00 +0000,wsj,"Trade attorneys, customs brokers and importers...",https://www.wsj.com/articles/here-are-some-ans...
141,53,2026-02-23-22-17-00 +0000,wsj,Rep. Tony Gonzales of Texas faced calls from R...,https://www.wsj.com/politics/policy/tony-gonza...
291,50,2026-02-23-15-34-59 +0000,latimes,Nick Reiner pleads not guilty to murder in kil...,https://www.latimes.com/california/story/2026-...
321,43,2026-02-23-12-08-15 +0000,nypost,Trump invites gold-winning US men’s Olympic ho...,https://nypost.com/2026/02/23/us-news/trump-in...
72,42,2026-02-24-01-13-00 +0000,wsj,Oil Edges Lower Amid Possible Position Adjustm...,https://www.wsj.com/finance/commodities-future...
296,39,2026-02-23-15-16-32 +0000,nypost,Federal judge blocks release of Jack Smith rep...,https://nypost.com/2026/02/23/us-news/judge-pe...
363,38,2026-02-23-08-37-41 +0000,nypost,Rocket launchers among military-grade equipmen...,https://nypost.com/2026/02/23/world-news/rocke...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
